In [1]:
import os
import tkinter as tk
from tkinter import simpledialog
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.action_chains import ActionChains


# Función para obtener credenciales
def get_credentials():
    root = tk.Tk()
    root.withdraw()  # Oculta la ventana principal

    # Solicita el correo electrónico
    email = simpledialog.askstring("Credenciales", "Introduce tu correo electrónico:")
    password = simpledialog.askstring("Credenciales", "Introduce tu contraseña:", show='*')

    return email, password
# Función para descargar el informe en PDF
def download_pdf(driver, wait):
    try:
        # Abre el menú de exportación
        export_menu_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "[data-testid='appbar-export-options-btn'] > span")))
        export_menu_btn.click()

        # Selecciona la opción de exportar a PDF
        export_to_pdf_btn = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@data-testid="export-to-pdf-btn"]')))
        export_to_pdf_btn.click()

        # Añade un tiempo de espera para asegurar que los elementos se carguen
        time.sleep(2)

        # Encuentra todas las etiquetas con el texto del checkbox
        checkbox_labels = driver.find_elements(By.XPATH, '//label')

        # (Opcional) Desmarcar la opción "Excluir pestañas del informe ocultas"
        for label in checkbox_labels:
            if "Exclude hidden report tabs" in label.text:
                checkbox = label.find_element(By.XPATH, 'preceding-sibling::input')
                if not checkbox.is_selected():
                    checkbox.click()
                break

        # Itera a través de las etiquetas y selecciona el checkbox correcto
        for label in checkbox_labels:
            if "Only export current page" in label.text:
                checkbox = label.find_element(By.XPATH, 'preceding-sibling::input')
                driver.execute_script("arguments[0].scrollIntoView(true);", checkbox)
                if not checkbox.is_selected():
                    driver.execute_script("arguments[0].click();", checkbox)
                break

        # Confirma la descarga
        ok_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@data-testid="dailog-ok-btn"]')))
        ok_button.click()

        print("Informe PDF descargándose...")

        # Verifica si el archivo se ha descargado
        current_directory = os.getcwd()
        filename = "nombre_del_archivo.pdf"  # Cambia esto al nombre esperado del archivo PDF
        if is_file_downloaded(current_directory, filename):
            print("La descarga del PDF se completó con éxito.")
        else:
            print("La descarga del PDF no se completó en el tiempo esperado.")

    except Exception as e:
        print(f"Error al descargar el PDF: {e}")

# Función para verificar si el archivo se ha descargado
def is_file_downloaded(directory, filename, timeout=60):
    start_time = time.time()
    file_path = os.path.join(directory, filename)
    
    while time.time() - start_time < timeout:
        if os.path.isfile(file_path) and os.path.getsize(file_path) > 0:
            # Verifica si el archivo está siendo descargado (tamaño cambia)
            initial_size = os.path.getsize(file_path)
            time.sleep(5)  # Espera unos segundos
            final_size = os.path.getsize(file_path)
            if initial_size == final_size:
                return True
    return False



# Configura las opciones de Firefox
options = Options()
options.add_argument("--headless")  # Ejecutar en modo headless si no quieres ver la UI
options.set_preference('general.useragent.override', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/74.0')
options.set_preference('network.proxy.type', 1)  # Configura el uso del proxy
options.set_preference('network.proxy.http', 'localhost')
options.set_preference('network.proxy.http_port', 8080)
options.set_preference('network.proxy.ssl', 'localhost')
options.set_preference('network.proxy.ssl_port', 8080)
options.set_preference('security.enterprise_roots.enabled', True)  # Ignorar los errores de certificados

# Usa el directorio actual como ruta de descarga
current_directory = os.getcwd()
options.set_preference("browser.download.folderList", 2)  # Permitir descarga en una ubicación específica
options.set_preference("browser.download.dir", current_directory)  # Establece el directorio actual como ruta de descarga
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf")

# Inicia el navegador
driver = webdriver.Firefox(options=options)
wait = WebDriverWait(driver, 30)

# Abre la página de Power BI
driver.get("https://app.powerbi.com/groups/e972c2cb-848c-4d27-b3b0-dcd2fc170e81/reports/82ef2309-602f-48d7-af7e-712cd9d464aa/ReportSectiond56ca7ee2eb409a0ffad?experience=power-bi&bookmarkGuid=Bookmarkb48a25697bac0cdc4290")

try:
    # Intenta detectar el inicio de sesión
    email_field = wait.until(EC.presence_of_element_located((By.ID, 'email')))
    if email_field:
        # Obtiene las credenciales
        email, password = get_credentials()
        email_field.send_keys(email)
        email_field.send_keys(Keys.RETURN)

        # Espera a que aparezca el campo de contraseña
        password_field = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[type="password"]')))
        password_field.send_keys(password)
        password_field.send_keys(Keys.RETURN)

        # Detecta la ventana de "¿Quiere mantener la sesión iniciada?"
        keep_signed_in = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[id="idBtn_Back"]')))
        keep_signed_in.click()  # Hace clic en "No"
        download_pdf(driver, wait)

except Exception as e:
    print(f"Ocurrió un error: {e}")

finally:
    # Cierra el navegador
    driver.quit()


Informe PDF descargándose...
La descarga del PDF no se completó en el tiempo esperado.


In [ ]:
# mitmdump -s mitm_scr_2.py "~h X-PowerBI"